In [9]:
import os
import csv
import pandas as pd
from google.colab import files

def use_existing_data():
    print("You chose to use existing data.")
    data_directory = "/content/drive/MyDrive/Colab"
    files = os.listdir(data_directory)

    print("Available files:")
    for i, file in enumerate(files):
        print(f"{i + 1}. {file}")

    choice = input("Enter the number of the file you want to use: ")

    try:
        choice = int(choice)
        if 1 <= choice <= len(files):
            selected_file = files[choice - 1]
            print(f"You selected: {selected_file}")
            data = pd.read_csv(os.path.join(data_directory, selected_file),sep=',')
            return data
        else:
            print("Invalid choice. Please enter a number within the range.")
    except ValueError:
        print("Invalid input. Please enter a number.")

def use_own_data():
    print("You chose to use your own data.")

    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')
        try:
            data = pd.read_csv(filename, sep=',')
            return data
        except Exception as e:
            print("Error processing the uploaded file:", e)

print("Welcome to the data selection menu!")
print("1. Use your own data")
print("2. Use existing data")

choice = input("Enter your choice (1 or 2): ")

if choice == '1':
    data = use_own_data()
    print("Data as DataFrame:")
    print(data)
elif choice == '2':
    data = use_existing_data()
    print("Data as DataFrame:")
    print(data)
else:
    print("Invalid choice. Please enter 1 or 2.")

Welcome to the data selection menu!
1. Use your own data
2. Use existing data
Enter your choice (1 or 2): 2
You chose to use existing data.
Available files:
1. None.csv
2. read_counts.csv
3. GSM2406675_10X001_cell_identities.csv
4. GSE224008_exp.csv
5. GSE103322.csv
6. GSE99611.csv
7. GSE254030.csv
8. Bulk_RNA.csv
9. micro.csv
10. LET_7A_2_3P.v2023.2.Hs.gmt
11. GOBP_MRNA_TRANSCRIPTION.v2023.2.Hs.gmt
Enter the number of the file you want to use: 8
You selected: Bulk_RNA.csv
Data as DataFrame:
    gene_name  GSM2648567  GSM2648568  GSM2648569  GSM2648570  GSM2648571  \
0      KCNAB2   11.773600   11.658110   11.782950   11.761680    11.49308   
1         PGD   12.091380   12.193080   12.094560   12.183170    12.22029   
2       RNU5E   10.445670   10.494380   10.481250   10.479750    10.29784   
3       PLOD1   10.688640   10.573110   10.686220   10.641750    10.52194   
4        MFN2   10.336410   10.173360   10.317230   10.303950    10.28382   
..        ...         ...         ...    

In [16]:
import pandas as pd
import numpy as np

def normalize_data(data):
  data_numeric = data.select_dtypes(include=[np.number])
  library_sizes = data_numeric.sum(axis=0)
  effective_library_size = np.exp(np.mean(np.log(library_sizes)))
  normalization_factors = library_sizes / effective_library_size
  normalized_data = data_numeric.div(normalization_factors, axis=1)
  return normalized_data

data_normalized = normalize_data(data)
print(data_normalized)


     GSM2648567  GSM2648568  GSM2648569  GSM2648570  GSM2648571  GSM2648572  \
0     11.783427   11.655171   11.796558   11.760004   11.470377   11.674179   
1     12.101473   12.190006   12.108528   12.181434   12.196150   12.212098   
2     10.454389   10.491734   10.493355   10.478257   10.277498   10.598301   
3     10.697562   10.570444   10.698561   10.640234   10.501155   10.559206   
4     10.345038   10.170795   10.329145   10.302482   10.263505   10.234472   
..          ...         ...         ...         ...         ...         ...   
669   10.371940   10.050515   10.385400   10.258758   10.149801   10.221774   
670   10.356397    9.996368   10.256692   10.154663   10.206618   10.018876   
671    9.985579   10.116369    9.906997    9.942138   10.007123    9.997480   
672    9.958487   10.156389   10.143801   10.085493   10.131986   10.115306   
673   10.315183   10.312989   10.413733   10.319969   10.383738   10.468389   

     GSM2648573  GSM2648574  GSM2648575  GSM2648576

In [17]:
from scipy import stats
import numpy as np


num_columns = len(data_normalized.columns)
group_A_columns = data_normalized.columns[:num_columns // 2]
group_B_columns = data_normalized.columns[num_columns // 2:]

t_test_results = {}
for gene in data_normalized.index:
    t_statistic, p_value = stats.ttest_ind(data_normalized.loc[gene, group_A_columns],
                                            data_normalized.loc[gene, group_B_columns])
    t_test_results[gene] = {'t_statistic': t_statistic, 'p_value': p_value}

significant_genes = {}
for gene, result in t_test_results.items():
    p_value = result['p_value']
    if isinstance(p_value, np.ndarray):
        p_value = p_value[0]
    if p_value < 0.05:
        significant_genes[gene] = result

significant_gene_names = []

for gene, result in t_test_results.items():
    p_value = result['p_value']
    if isinstance(p_value, np.ndarray):
        p_value = p_value[0]
    if p_value < 0.05:
        significant_gene_names.append(gene)
print("Significant gene names:")
print(significant_gene_names)
print(len(significant_gene_names))


Significant gene names:
[1, 3, 13, 14, 17, 30, 31, 37, 41, 52, 53, 54, 55, 59, 60, 62, 63, 65, 66, 70, 84, 86, 88, 89, 90, 93, 95, 97, 104, 108, 111, 115, 117, 122, 126, 131, 132, 133, 134, 136, 142, 143, 150, 154, 156, 158, 160, 166, 177, 182, 185, 187, 188, 189, 190, 192, 209, 215, 216, 217, 218, 221, 228, 230, 231, 235, 241, 243, 252, 254, 256, 258, 259, 260, 269, 272, 276, 279, 280, 290, 295, 296, 298, 299, 301, 310, 312, 314, 315, 319, 320, 321, 322, 325, 326, 330, 332, 341, 344, 345, 351, 352, 353, 354, 356, 359, 369, 374, 379, 381, 384, 389, 390, 397, 400, 403, 408, 415, 416, 419, 421, 424, 427, 437, 438, 446, 448, 453, 457, 462, 468, 470, 487, 490, 492, 496, 500, 503, 507, 517, 518, 521, 531, 532, 533, 540, 544, 563, 569, 570, 579, 584, 587, 591, 592, 595, 597, 598, 601, 602, 603, 608, 616, 620, 623, 639, 644, 652, 672, 673]
170


In [18]:
genes_per_set = 3

gene_sets = {}
current_set_index = 1
for i in range(0, len(significant_gene_names), genes_per_set):
    gene_set_name = f'Gene_Set_{current_set_index}'
    gene_sets[gene_set_name] = significant_gene_names[i:i+genes_per_set]
    current_set_index += 1

top_gene_sets = dict(list(gene_sets.items())[:10])

print("Top 10 Gene Sets:")
for gene_set_name, genes in top_gene_sets.items():
    print(f"{gene_set_name}: {genes}")



Top 10 Gene Sets:
Gene_Set_1: [1, 3, 13]
Gene_Set_2: [14, 17, 30]
Gene_Set_3: [31, 37, 41]
Gene_Set_4: [52, 53, 54]
Gene_Set_5: [55, 59, 60]
Gene_Set_6: [62, 63, 65]
Gene_Set_7: [66, 70, 84]
Gene_Set_8: [86, 88, 89]
Gene_Set_9: [90, 93, 95]
Gene_Set_10: [97, 104, 108]


In [19]:
gene_ranks = {gene: result['t_statistic'] for gene, result in t_test_results.items()}

gsea_results = {}
for gene_set_name, gene_set in gene_sets.items():
    enrichment_scores = [gene_ranks[gene] if gene in gene_ranks else 0 for gene in gene_set]
    enrichment_score = sum(enrichment_scores)
    gsea_results[gene_set_name] = enrichment_score

sorted_gsea_results = sorted(gsea_results.items(), key=lambda x: np.any(x[1]), reverse=True)

print("Top 10 Gene Set Enrichment Analysis Results:")
for gene_set_name, enrichment_score in sorted_gsea_results[:10]:
    print(f"{gene_set_name}: {enrichment_score}")

Top 10 Gene Set Enrichment Analysis Results:
Gene_Set_1: -4.24369027035398
Gene_Set_2: -7.804650988568008
Gene_Set_3: -1.3268170660782759
Gene_Set_4: 5.875668368478719
Gene_Set_5: -9.434482698123094
Gene_Set_6: -3.213831448554857
Gene_Set_7: -8.547198403865819
Gene_Set_8: 8.517027836775535
Gene_Set_9: 2.3271472722374393
Gene_Set_10: -1.8929671246588469


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
